# Causal / Anti Causal 상황에 따른 Semi-Supervised Learning의 성능 비교(TSVM/SVM)

### *Causal 상황 : 설명변수들이 Cause, 반응변수가 Effect
### *Anti Causal 상황: 설명변수들이 Effect, 반응변수가 Cause
### *mixed 상황 : 일부 설명변수는 반응변수의 Cause, 그 외 설명변수는 반응변수의 Effect

### => 일반적으로 Causal 상황에서는 semi supervised learning이 supervised learning 보다 성능이 좋아지지 않음. 이는 추가적인 설명변수들의 분포에 대한 정보가 반응변수를 예측하는데 도움이 되지 않기 때문임.

### =>여기에서는 이를 실제 데이터 분석을 통해 보이고자 함. (예측성능에 대한 평가는 임의로 설정한 test set에서의 MSE로 하였음.)

## 1. Mushroom Data

In [1]:
setwd("C:/Users/MinSu/Desktop/Project-master/Project-master/Semi_supervised_learning/data")

In [2]:
df1=read.table("agaricus-lepiota.data",sep=",")
head(df1)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [3]:
table(df1$V12)


   ?    b    c    e    r 
2480 3776  556 1120  192 

In [4]:
df1=subset(df1,V12!="?") #설명변수 V12, 식별되지 않은 ?는 제거

In [5]:
df1$V12=factor(df1$V12)
str(df1)

'data.frame':	5644 obs. of  23 variables:
 $ V1 : Factor w/ 2 levels "e","p": 2 1 1 2 1 1 1 1 2 1 ...
 $ V2 : Factor w/ 6 levels "b","c","f","k",..: 6 6 1 6 6 6 1 1 6 1 ...
 $ V3 : Factor w/ 4 levels "f","g","s","y": 3 3 3 4 3 4 3 4 4 3 ...
 $ V4 : Factor w/ 10 levels "b","c","e","g",..: 5 10 9 9 4 10 9 9 9 10 ...
 $ V5 : Factor w/ 2 levels "f","t": 2 2 2 2 1 2 2 2 2 2 ...
 $ V6 : Factor w/ 9 levels "a","c","f","l",..: 7 1 4 7 6 1 1 4 7 1 ...
 $ V7 : Factor w/ 2 levels "a","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ V8 : Factor w/ 2 levels "c","w": 1 1 1 1 2 1 1 1 1 1 ...
 $ V9 : Factor w/ 2 levels "b","n": 2 1 1 2 1 1 1 1 2 1 ...
 $ V10: Factor w/ 12 levels "b","e","g","h",..: 5 5 6 6 5 6 3 6 8 3 ...
 $ V11: Factor w/ 2 levels "e","t": 1 1 1 1 2 1 1 1 1 1 ...
 $ V12: Factor w/ 4 levels "b","c","e","r": 3 2 2 3 3 2 2 2 3 2 ...
 $ V13: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V14: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V15: Factor w/ 9 levels "b","c","e

In [6]:
df1=df1[,-17] #V17은 한개의 값으로 되어있으므로 변수의 역할을 못하므로 삭제.

In [7]:
setnum=sample(1:5644,1000) #Numerical study의 편의상 샘플 1000개를 랜덤샘플링함.
df1=df1[setnum,]

In [8]:
df1[1,] #첫 행

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V13,V14,V15,V16,V18,V19,V20,V21,V22,V23
1233,e,f,f,g,f,n,f,w,b,k,...,f,s,w,w,w,o,e,k,a,g


In [11]:
#첫번째 Validation
tp=1:700
testdata=df1[tp,] #test set, 관측치 700개
traindata=df1[-tp,] 

In [12]:
l_num=sample(1:300,100)
traindata_l=traindata[l_num,] #train set(X_l), 반응변수,설명변수 모두 있는 관측치는 100개
traindata_u=traindata[-l_num,]
traindata_u$V1=rep(NA,200)     #train set(X_u), 설명변수만 있는 관측치는 200개

In [13]:
X <- traindata_l[,2:22]
X_u <- traindata_u[1:200,2:22] #설명변수만 있는 관측치들 (Supervised Learining 입장에서는 덤으로 있는 관측치들)
y <- traindata_l[,1]
Xe<-rbind(X,X_u)

In [14]:
library("RSSL")
g_sup <- SVM(X,y,x_center=FALSE,scale=FALSE) #Supervised Learning
g_constraint <- TSVM(X=X,y=y,X_u=X_u,
                     C=10,Cstar=10,balancing_constraint = TRUE,
                     x_center = FALSE,verbose=TRUE) #Semi Supervised Learning

Loss @ Start:  2155.116 
w    @ Start:  0.2405907 0 -0.1198697 -0.1345305 0 0.01380942 0 -0.4116851 0.1113133 0.1986055 0.0656416 0.1654288 0.2392477 -0.228077 0 0 0.08667902 0.07459449 0.1755324 -0.4705394 -0.5117361 0.3871176 0 0.355302 -0.5212024 0 0 1.665335e-16 0.2719474 -0.6139594 0 0.03648331 -0.08284957 0.2511801 0.207527 0 -0.08104459 0 -0.1757068 0.08381716 -0.2394066 0.41846 0.5409772 -0.1564979 0.05408474 -0.1337726 0.1257332 -0.2394066 -0.1337726 0.219011 -0.1853219 0 0 0.05237025 -1.684547e-13 0 0.05749524 0.2551673 -0.2394066 0 0 0.07175823 0.1778554 0 1.884765e-12 -0.002060716 -0.2394066 0 0.2394066 -0.2394066 0.1914894 -0.1914894 0 -0.1337726 0 -0.2950069 -0.5117361 0.1770418 0.1484013 0 -0.4845741 0.3137072 0.3571598 0 -0.2394066 0.06495723 -0.08300809 -0.1093817 0.2049437 0.1123406 0.06407506 -0.195961 0.3015379 -0.117546 0 -0.5966072 
Loss @ iteration  0 :  3953.546 
w    @ iteration  0 :  0.2020806 0 0.2683687 -0.09487035 0 0.7330803 8.959766e-15 -0.1143038 0.36911

In [15]:
X_test=testdata[,2:22]
mean(testdata$V1==predict(g_sup,X_test))
mean(testdata$V1==predict(g_constraint,X_test)) #성능이 살짝 좋아짐..

[1] 0.9785714

[1] 0.9814286